# ENSF 400 - Winter 2024 - Lab 03 - Docker


## Install Docker and Docker Compose on your laptop

Follow the instructions on [Install Docker Desktop](https://docs.docker.com/desktop/install/mac-install/). Find the guide for your operating system (Windows, Mac, Linux) on the left side of the webpage.

After the installation is complete, test out the commands we are going to use:
```
$ docker -v
Docker version 24.0.2, build cb74dfc

```


Test if docker command works as expected:
```
$ docker ps
CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES
```


## First Alpine Linux Containers

In this lab you will run a popular, free, lightweight container and explore the basics of how containers work, how the Docker Engine executes and isolates containers from each other. If you already have experience running containers and basic Docker commands you can probably skip this intro exercise.

------

Concepts in this exercise:

- Docker engine
- Containers & images
- Image registries (AKA Docker Hub)
- Container isolation

------

Tips:

Code snippets are shown in one of three ways throughout this environment:

1. Code that looks like `this` is sample code snippets that is usually part of an explanation.

2. Code that appears in box like the one below can be clicked on and it will automatically be typed in to the appropriate terminal window:

   ```.term1
   uname -a
   ```

3. Code appearing in windows like the one below is code that you should type in yourself. Usually there will be a unique ID or other bit your need to enter which we cannot supply. Items appearing in <> are the pieces you should substitute based on the instructions.

   ```bash
   docker start <container ID>
   ```

### Running your first container

It’s time to get your hands dirty! As with all things technical, a “hello world” app is good place to start. Type or click the code below to run your first Docker container:

```.term1
docker run hello-world
```

That’s it: your first container. The *hello-world* container output tells you a bit about what just happened. Essentially, the Docker engine running in your terminal tried to find an **image** named hello-world. Since you just got started there are no images stored locally (`Unable to find image...`) so Docker engine goes to its default **Docker registry**, which is [Docker Store](https://store.docker.com/), to look for an image named “hello-world”. It finds the image there, pulls it down, and then runs it in a container. And hello-world’s only function is to output the text you see in your terminal, after which the container exits.

![Hello world explainer](https://training.play-with-docker.com/images/ops-basics-hello-world.svg)

If you are familiar with VMs, you may be thinking this is pretty much just like running a virtual machine, except with a central repository of VM images. And in this simple example, that is basically true. But as you go through these exercises you will start to see important ways that Docker and containers differ from VMs. For now, the simple explanation is this:

- The VM is a *hardware* abstraction: it takes physical CPUs and RAM from a host, and divides and shares it across several smaller virtual machines. There is an OS and application running inside the VM, but the virtualization software usually has no real knowledge of that.
- A container is an *application* abstraction: the focus is really on the OS and the application, and not so much the hardware abstraction. Many customers actually use both VMs and containers today in their environments and, in fact, may run containers inside of VMs.

### Docker Images

In this rest of this lab, you are going to run an [Alpine Linux](http://www.alpinelinux.org/) container. Alpine is a lightweight Linux distribution so it is quick to pull down and run, making it a popular starting point for many other images.

To get started, let’s run the following in our terminal:

```.term1
docker image pull alpine
```

The `pull` command fetches the alpine **image** from the **Docker registry** and saves it in our system.

You can use the `docker image` command to see a list of all images on your system.

```.term1
docker image ls
```

```
REPOSITORY              TAG                 IMAGE ID            CREATED             VIRTUAL SIZE
alpine                 latest              c51f86c28340        4 weeks ago         1.109 MB
hello-world             latest              690ed74de00f        5 months ago        960 B
```

### Docker Run

Great! Let’s now run a Docker **container** based on this image. To do that you are going to use the `docker run` command.

```
docker run alpine ls -l
```

```.term1
total 48
drwxr-xr-x    2 root     root          4096 Mar  2 16:20 bin
drwxr-xr-x    5 root     root           360 Mar 18 09:47 dev
drwxr-xr-x   13 root     root          4096 Mar 18 09:47 etc
drwxr-xr-x    2 root     root          4096 Mar  2 16:20 home
drwxr-xr-x    5 root     root          4096 Mar  2 16:20 lib
......
......

```

While the output of the `ls` command may not be all that exciting, behind the scenes quite a few things just took place. When you call `run`, the Docker client finds the image (alpine in this case), creates the container and then runs a command in that container. When you run `docker run alpine`, you provided a command (`ls -l`), so Docker executed this command inside the container for which you saw the directory listing. After the `ls` command finished, the container shut down.

![docker run explainer](https://training.play-with-docker.com/images/ops-basics-run-details.svg)

The fact that the container exited after running our command is important, as you will start to see. Let’s try something more exciting. Type in the following:

```.term1
docker container run alpine echo "hello from alpine"
```

And you should get the following output:

```
hello from alpine
```

In this case, the Docker client dutifully ran the `echo` command inside our alpine container and then exited. If you noticed, all of that happened pretty quickly and again our container exited. As you will see in a few more steps, the `echo` command ran in a separate container instance. Imagine booting up a virtual machine (VM), running a command and then killing it; it would take a minute or two just to boot the VM before running the command. A VM has to emulate a full hardware stack, boot an operating system, and then launch your app - it’s a virtualized *hardware* environment. Docker containers function at the application layer so they skip most of the steps VMs require and just run what is required for the app. Now you know why they say containers are fast!

Try another command.

```.term1
docker run alpine /bin/sh
```

Wait, nothing happened! Is that a bug? No! In fact, something did happen. You started a 3rd instance of the alpine container and it ran the command `/bin/sh` and then exited. You did not supply any additional commands to `/bin/sh` so it just launched the shell, exited the shell, and then stopped the container. What you might have *expected* was an interactive shell where you could type some commands. Docker has a facility for that by adding a flag to run the container in an interactive terminal. For this example, type the following:

```.term1
docker run -it alpine /bin/sh
```

You are now inside the container running a Linux shell and you can try out a few commands like `ls -l`, `uname -a` and others. Note that Alpine is a small Linux OS so several commands might be missing. Exit out of the shell and container by typing the `exit` command.

Ok, we said that we had run each of our commands above in a separate container instance. We can see these instances using the `docker ps` command. The `docker container ls` command by itself shows you all containers that are currently running:

```.term1
docker ps
```

```
CONTAINER ID        IMAGE               COMMAND                  CREATED             STATUS                      PORTS               NAMES
36171a5da744        alpine              "/bin/sh"                5 minutes ago       Exited (0) 2 minutes ago                        fervent_newton
a6a9d46d0b2f        alpine             "echo 'hello from alp"    6 minutes ago       Exited (0) 6 minutes ago                        lonely_kilby
ff0a5c3750b9        alpine             "ls -l"                   8 minutes ago       Exited (0) 8 minutes ago                        elated_ramanujan
c317d0a9e3d2        hello-world         "/hello"                 34 seconds ago      Exited (0) 12 minutes ago                       stupefied_mcclintock
```

Since no containers are running, you see a blank line. Let’s try a more useful variant: `docker container ls -a`

```.term1
docker ps -a
```

What you see now is a list of all containers that you ran. Notice that the `STATUS` column shows that these containers exited some time ago.

Here is the same output of the `docker ps -a` command, shown diagrammatically (note that your container IDs and names will be different):

![Docker container instances](https://training.play-with-docker.com/images/ops-basics-instances.svg)

It makes sense to spend some time getting comfortable with the `docker run` commands. To find out more about `run`, use `docker run --help` to see a list of all flags it supports. As you proceed further, we’ll see a few more variants of `docker run` but feel free to experiment here before proceeding.

### Container Isolation

In the steps above we ran several commands via container instances with the help of `docker container run`. The `docker ps -a` command showed us that there were several containers listed. Why are there so many containers listed if they are all from the *alpine* image?

This is a critical security concept in the world of Docker containers! Even though each `docker run` command used the same alpine **image**, each execution was a separate, isolated **container**. Each container has a separate filesystem and runs in a different namespace; by default a container has no way of interacting with other containers, even those from the same image. Let’s try another exercise to learn more about isolation.

```.term1
docker run -it alpine /bin/ash
```

The `/bin/ash` is another type of shell available in the alpine image. Once the container launches and you are at the container’s command prompt type the following commands:

```
echo "hello world" > hello.txt

ls
```

The first `echo` command creates a file called “hello.txt” with the words “hello world” inside it. The second command gives you a directory listing of the files and should show your newly created “hello.txt” file. Now type `exit` to leave this container.

To show how isolation works, run the following:

```.term1
docker run alpine ls
```

It is the same `ls` command we used inside the container’s interactive ash shell, but this time, did you notice that your “hello.txt” file is missing? That’s isolation! Your command ran in a new and separate *instance*, even though it is based on the same *image*. The 2nd instance has no way of interacting with the 1st instance because the Docker Engine keeps them separated and we have not setup any extra parameters that would enable these two instances to interact.

In every day work, Docker users take advantage of this feature not only for security, but to test the effects of making application changes. Isolation allows users to quickly create separate, isolated test copies of an application or service and have them run side-by-side without interfering with one another. In fact, there is a whole lifecycle where users take their changes and move them up to production using this basic concept and the built-in capabilities of Docker Enteprise. We will explore more of that in later exercises.

Right now, the obvious question is “how do I get back to the container that has my ‘hello.txt’ file?”

Once again run the

```.term1
docker run alpine ls -a
```

command again and you should see output similar to the following:

```
CONTAINER ID        IMAGE               COMMAND                  CREATED             STATUS                      PORTS               NAMES
36171a5da744        alpine              "ls"                     2 minutes ago       Exited (0) 2 minutes ago                        distracted_bhaskara
3030c9c91e12        alpine              "/bin/ash"               5 minutes ago       Exited (0) 2 minutes ago                        fervent_newton
a6a9d46d0b2f        alpine             "echo 'hello from alp"    6 minutes ago       Exited (0) 6 minutes ago                        lonely_kilby
ff0a5c3750b9        alpine             "ls -l"                   8 minutes ago       Exited (0) 8 minutes ago                        elated_ramanujan
c317d0a9e3d2        hello-world         "/hello"                 34 seconds ago      Exited (0) 12 minutes ago                       stupefied_mcclintock
```

Graphically this is what happened on our Docker Engine: ![Docker container isolation](https://training.play-with-docker.com/images/ops-basics-isolation.svg)

The container in which we created the “hello.txt” file is the same one where we used the `/bin/ash` shell, which we can see listed in the “COMMAND” column. The *Container ID* number from the first column uniquely identifies that particular container instance. In the sample output above the container ID is `3030c9c91e12`. We can use a slightly different command to tell Docker to run this specific container instance. Try typing:

```
docker start <container ID>
```

- **Pro tip:** Instead of using the full container ID you can use just the first few characters, as long as they are enough to uniquely ID a container. So we could simply use “3030” to identify the container instance in the example above, since no other containers in this list start with these characters.

Now use the `docker container ls` command again to list the running containers.

```
CONTAINER ID        IMAGE               COMMAND                  CREATED             STATUS                      PORTS               NAMES
3030c9c91e12        alpine              "/bin/ash"                2 minutes ago       Up 14 seconds                        distracted_bhaskara
```

Notice this time that our container instance is still running. We used the ash shell this time so the rather than simply exiting the way /bin/sh did earlier, ash waits for a command. We can send a command in to the container to run by using the `exec` command, as follows:

```
docker exec <container ID> ls
```

This time we get a directory listing and it shows our “hello.txt” file because we used the container instance where we created that file.

![Docker container exec command](https://training.play-with-docker.com/images/ops-basics-exec.svg)

Now you are starting to see some of the important concepts of containers. In the next exercise we will start to see how you can create your own Docker images and how to use a Dockerfile to standardize images such that you can create larger, more complex images in a simple, automated manner.

### Terminology

In the last section, you saw a lot of Docker-specific jargon which might be confusing to some. So before you go further, let’s clarify some terminology that is used frequently in the Docker ecosystem.

- *Images* - The file system and configuration of our application which are used to create containers. To find out more about a Docker image, run `docker image inspect alpine`. In the demo above, you used the `docker image pull` command to download the **alpine** image. When you executed the command `docker run hello-world`, it also did a `docker image pull` behind the scenes to download the **hello-world** image.
- *Containers* - Running instances of Docker images — containers run the actual applications. A container includes an application and all of its dependencies. It shares the kernel with other containers, and runs as an isolated process in user space on the host OS. You created a container using `docker run` which you did using the alpine image that you downloaded. A list of running containers can be seen using the `docker ps` command.
- *Docker daemon* - The background service running on the host that manages building, running and distributing Docker containers.
- *Docker client* - The command line tool that allows the user to interact with the Docker daemon.
- *Docker Store* - Store is, among other things, a [registry](https://store.docker.com/) of Docker images. You can think of the registry as a directory of all available Docker images. You’ll be using this later in this tutorial.


## Image creation from a container

Let’s start by running an interactive shell in a ubuntu container:

```.term1
docker run -ti ubuntu bash
```

As you know from earlier labs, you just grabbed the image called “ubuntu” from Docker Store and are now running the bash shell inside that container.[1](https://training.play-with-docker.com/ops-s1-images/#fn:1)

To customize things a little bit we will install a package called [figlet](http://www.figlet.org/) in this container. Your container should still be running so type the following commands at your ubuntu container command line:

```.term1
apt-get update
apt-get install -y figlet
figlet "hello docker"
```

You should see the words “hello docker” printed out in large ascii characters on the screen. Go ahead and exit from this container

```.term1
exit
```

Now let us pretend this new figlet application is quite useful and you want to share it with the rest of your team. You *could* tell them to do exactly what you did above and install figlet in to their own container, which is simple enough in this example. But if this was a real world application where you had just installed several packages and run through a number of configuration steps the process could get cumbersome and become quite error prone. Instead, it would be easier to create an *image* you can share with your team.

To start, we need to get the ID of this container using the ls command (do not forget the -a option as the non running container are not returned by the ls command).

```.term1
docker ps -a
```

Before we create our own image, we might want to inspect all the changes we made. Try typing the command `docker diff <container ID>` for the container you just created. You should see a list of all the files that were added or changed to in the container when you installed figlet. Docker keeps track of all of this information for us. This is part of the *layer* concept we will explore in a few minutes.

Now, to create an image we need to “commit” this container. Commit creates an image locally on the system running the Docker engine. Run the following command, using the container ID you retrieved, in order to commit the container and create an image out of it.

```
docker commit CONTAINER_ID
```

That’s it - you have created your first image! Once it has been commited, we can see the newly created image in the list of available images.

```.term1
docker image ls
```

You should see something like this:

```
REPOSITORY          TAG                 IMAGE ID            CREATED             SIZE
<none>              <none>              a104f9ae9c37        46 seconds ago      160MB
ubuntu              latest              14f60031763d        4 days ago          120MB
```

Note that the image we pulled down in the first step (ubuntu) is listed here along with our own custom image. Except our custom image has no information in the REPOSITORY or TAG columns, which would make it tough to identify exactly what was in this container if we wanted to share amongst multiple team members.

Adding this information to an image is known as *tagging* an image. From the previous command, get the ID of the newly created image and tag it so it’s named **ourfiglet**:

```
docker image tag <IMAGE_ID> ourfiglet
docker image ls
```

Now we have the more friendly name “ourfiglet” that we can use to identify our image.

```
REPOSITORY          TAG                 IMAGE ID            CREATED             SIZE
ourfiglet           latest              a104f9ae9c37        5 minutes ago       160MB
ubuntu              latest              14f60031763d        4 days ago          120MB
```

Here is a graphical view of what we just completed: ![commit container to image](https://training.play-with-docker.com/images/ops-images-commit.svg)

Now we will run a container based on the newly created *ourfiglet* image:

```.term1
docker run ourfiglet figlet hello
```

As the figlet package is present in our *ourfiglet* image, the command returns the following output:

```
 _          _ _
| |__   ___| | | ___
| '_ \ / _ \ | |/ _ \
| | | |  __/ | | (_) |
|_| |_|\___|_|_|\___/
```

This example shows that we can create a container, add all the libraries and binaries in it and then commit it in order to create an image. We can then use that image just as we would for images pulled down from the Docker Store. We still have a slight issue in that our image is only stored locally. To share the image we would want to *push* the image to a registry somewhere. This is beyond the scope of this lab (and you should not enter any personal login information in these labs) but you can get a free Docker ID, run these labs, and push to the [Docker Community Hub](https://hub.docker.com/) from your own system using [Docker for Windows](https://www.docker.com/docker-windows) or [Docker for Mac](https://www.docker.com/docker-mac) if you want to try this out.

As mentioned above, this approach of manually installing software in a container and then committing it to a custom image is just one way to create an image. It works fine and is quite common. However, there is a more powerful way to create images. In the following exercise we will see how images are created using a *Dockerfile*, which is a text file that contains all the instructions to build an image.

## Image creation using a Dockerfile

Instead of creating a static binary image, we can use a file called a *Dockerfile* to create an image. The final result is essentially the same, but with a Dockerfile we are supplying the instructions for building the image, rather than just the raw binary files. This is useful because it becomes much easier to manage changes, especially as your images get bigger and more complex.

For example, if a new version of figlet is released we would either have to re-create our image from scratch, or run our image and upgrade the installed version of figlet. In contrast, a Dockerfile would include the `apt-get` commands we used to install figlet so that we - or anybody using the Dockerfile - could simply recompose the image using those instructions.

It is kind of like the old adage:

> *Give a sysadmin an image and their app will be up-to-date for a day, give a sysadmin a Dockerfile and their app will always be up-to-date*.

Ok, maybe that’s a bit of a stretch but Dockerfiles are powerful because they allow us to manage *how* an image is built, rather than just managing binaries. In practice, Dockerfiles can be managed the same way you might manage source code: they are simply text files so almost any version control system can be used to manage Dockerfiles over time.

We will use a simple example in this section and build a “hello world” application in Node.js. Do not be concerned if you are not familiar with Node.js: Docker (and this exercise) does not require you to know all these details.

We will start by creating a file in which we retrieve the hostname and display it. NOTE: You should be at the Docker host’s command line (`$`). If you see a command line that looks similar to `root@abcd1234567:/#` then you are probably still inside your ubuntu container from the previous exercise. Type `exit` to return to the host command line.

Type the following content into a file named *index.js*. You can use vi, vim or several other Linux editors in this exercise. If you need assistance with the Linux editor commands to do this follow this footnote[2](https://training.play-with-docker.com/ops-s1-images/#fn:2).

```
var os = require("os");
var hostname = os.hostname();
console.log("hello from " + hostname);
```

The file we just created is the javascript code for our server. As you can probably guess, Node.js will simply print out a “hello” message. We will Docker-ize this application by creating a Dockerfile. We will use **alpine** as the base OS image, add a Node.js runtime and then copy our source code in to the container. We will also specify the default command to be run upon container creation.

Create a file named *Dockerfile* and copy the following content into it. Again, help creating this file with Linux editors is here [3](https://training.play-with-docker.com/ops-s1-images/#fn:3).

```
FROM alpine
RUN apk update && apk add nodejs
COPY . /app
WORKDIR /app
CMD ["node","index.js"]
```

Let’s build our first image out of this Dockerfile and name it *hello:v0.1*:

```.term1
docker image build -t hello:v0.1 .
```

This is what you just completed: ![build container from dockerfile](https://training.play-with-docker.com/images/ops-images-dockerfile.svg)

We then start a container to check that our applications runs correctly:

```.term1
docker run hello:v0.1
```

You should then have an output similar to the following one (the ID will be different though).

```
hello from 92d79b6de29f
```

**What just happened?** We created two files: our application code (index.js) is a simple bit of javascript code that prints out a message. And the Dockerfile is the instructions for Docker engine to create our custom container. This Dockerfile does the following:

1. Specifies a base image to pull **FROM** - the *alpine* image we used in earlier labs.
2. Then it **RUN**s two commands (*apk update* and *apk add*) inside that container which installs the Node.js server.
3. Then we told it to **COPY** files from our working directory in to the container. The only file we have right now is our *index.js*.
4. Next we specify the **WORKDIR** - the directory the container should use when it starts up
5. And finally, we gave our container a command (**CMD**) to run when the container starts.

Recall that in previous labs we put commands like `echo "hello world"` on the command line. With a Dockerfile we can specify precise commands to run for everyone who uses this container. Other users do not have to build the container themselves once you push your container up to a repository (which we will cover later) or even know what commands are used. The *Dockerfile* allows us to specify *how* to build a container so that we can repeat those steps precisely everytime and we can specify *what* the container should do when it runs. There are actually multiple methods for specifying the commands and accepting parameters a container will use, but for now it is enough to know that you have the tools to create some pretty powerful containers.

## Image layers

There is something else interesting about the images we build with Docker. When running they appear to be a single OS and application. But the images themselves are actually built in **layers**. If you scroll back and look at the output from your `docker image build` command you will notice that there were 5 steps and each step had several tasks. You should see several “fetch” and “pull” tasks where Docker is grabbing various bits from Docker Store or other places. These bits were used to create one or more container *layers*. Layers are an important concept. To explore this, we will go through another set of exercises.

First, check out the image you created earlier by using the *history* command (remember to use the `docker image ls` command from earlier exercises to find your image IDs):

```
docker history <image ID>
```

What you see is the list of intermediate container images that were built along the way to creating your final Node.js app image. Some of these intermediate images will become *layers* in your final container image. In the history command output, the original Alpine layers are at the bottom of the list and then each customization we added in our Dockerfile is its own step in the output. This is a powerful concept because it means that if we need to make a change to our application, it may only affect a single layer! To see this, we will modify our app a bit and create a new image.

Type the following in to your console window:

```.term1
echo "console.log(\"this is v0.2\");" >> index.js
```

This will add a new line to the bottom of your *index.js* file from earlier so your application will output one additional line of text. Now we will build a new image using our updated code. We will also tag our new image to mark it as a new version so that anybody consuming our images later can identify the correct version to use:

```.term1
docker build -t hello:v0.2 .
```

You should see output similar to this:

```
Sending build context to Docker daemon  86.15MB
Step 1/5 : FROM alpine
 ---> 7328f6f8b418
Step 2/5 : RUN apk update && apk add nodejs
 ---> Using cache
 ---> 2707762fca63
Step 3/5 : COPY . /app
 ---> 07b2e2127db4
Removing intermediate container 84eb9c31320d
Step 4/5 : WORKDIR /app
 ---> 6630eb76312c
Removing intermediate container ee6c9e7a5337
Step 5/5 : CMD node index.js
 ---> Running in e079fb6000a3
 ---> e536b9dadd2f
Removing intermediate container e079fb6000a3
Successfully built e536b9dadd2f
Successfully tagged hello:v0.2
```

Notice something interesting in the build steps this time. In the output it goes through the same five steps, but notice that in some steps it says **Using cache**.

![layers and cache](https://training.play-with-docker.com/images/ops-images-cache.svg)

Docker recognized that we had already built some of these layers in our earlier image builds and since nothing had changed in those layers it could simply use a cached version of the layer, rather than pulling down code a second time and running those steps. Docker’s layer management is very useful to IT teams when patching systems, updating or upgrading to the latest version of code, or making configuration changes to applications. Docker is intelligent enough to build the container in the most efficient way possible, as opposed to repeatedly building an image from the ground up each and every time.

## Image Inspection

Now let us reverse our thinking a bit. What if we get a container from Docker Store or another registry and want to know a bit about what is inside the container we are consuming? Docker has an **inspect** command for images and it returns details on the container image, the commands it runs, the OS and more.

The *alpine* image should already be present locally from the exercises above (use `docker image ls` to confirm), if it’s not, run the following command to pull it down:

```.term1
docker pull alpine
```

Once we are sure it is there let’s inspect it.

```.term1
docker inspect alpine
```

There is a lot of information in there:

- the layers the image is composed of
- the driver used to store the layers
- the architecture / OS it has been created for
- metadata of the image
- …

We will not go into all the details here but we can use some filters to just inspect particular details about the image. You may have noticed that the image information is in JSON format. We can take advantage of that to use the inspect command with some filtering info to just get specific data from the image.

Let’s get the list of layers:

```.term1
docker inspect --format "{{ json .RootFS.Layers }}" alpine
```

Alpine is just a small base OS image so there’s just one layer:

```
["sha256:60ab55d3379d47c1ba6b6225d59d10e1f52096ee9d5c816e42c635ccc57a5a2b"]
```

Now let’s look at our custom Hello image. You will need the image ID (use `docker image ls` if you need to look it up):

```
docker inspect --format "{{ json .RootFS.Layers }}" <image ID>
```

Our Hello image is a bit more interesting (your sha256 hashes will vary):

```
["sha256:5bef08742407efd622d243692b79ba0055383bbce12900324f75e56f589aedb0","sha256:5ac283aaea742f843c869d28bbeaf5000c08685b5f7ba01431094a207b8a1df9","sha256:2ecb254be0603a2c76880be45a5c2b028f6208714aec770d49c9eff4cbc3cf25"]
```

We have three layers in our application. Recall that we had the base Alpine image (the FROM command in our Dockerfile), then we had a RUN command to install some packages, then we had a COPY command to add in our javascript code. Those are our layers! If you look closely, you can even see that both *alpine* and *hello* are using the same base layer, which we know because they have the same sha256 hash.

The tools and commands we explored in this lab are just the beginning. Docker Enterprise Edition includes private Trusted Registries with Security Scanning and Image Signing capabilities so you can further inspect and authenticate your images. In addition, there are policy controls to specify which users have access to various images, who can push and pull images, and much more.

Another important note about layers: each layer is immutable. As an image is created and successive layers are added, the new layers keep track of the changes from the layer below. When you start the container running there is an additional layer used to keep track of any changes that occur as the application runs (like the “hello.txt” file we created in the earlier exercises). This design principle is important for both security and data management. If someone mistakenly or maliciously changes something in a running container, you can very easily revert back to its original state because the base layers cannot be changed. Or you can simply start a new container instance which will start fresh from your pristine image. And applications that create and store data (databases, for example) can store their data in a special kind of Docker object called a **volume**, so that data can persist and be shared with other containers. We will explore volumes in a later lab.

Up next, we will look at more sophisticated applications that run across several containers and use Docker Compose and Docker Swarm to define our architecture and manage it.

## Terminology

- *Layers* - A Docker image is built up from a series of layers. Each layer represents an instruction in the image’s Dockerfile. Each layer except the last one is read-only.
- *Dockerfile* - A text file that contains all the commands, in order, needed to build a given image. The [Dockerfile reference](https://docs.docker.com/engine/reference/builder) page lists the various commands and format details for Dockerfiles.
- *Volumes* - A special Docker container layer that allows data to persist and be shared separately from the container itself. Think of volumes as a way to abstract and manage your persistent data separately from the application itself.

## Footnotes

1. A note on images and the public Docker Store (AKA Docker Hub): Docker registries are subdivided in to many *repositories*. This is the same for both our public registries like Docker Store / Docker Hub, as well as Docker Trusted Registries that you might run in your own environment. Image names must be unique and are specified in the format `<repository>/<image>:<tag>`. In our exercises, we pulled images called “ubuntu” and “alpine”. Since there is no repository specified we pulled from a default public repository called “library” which is maintained by us at Docker. And since we did not specify a tag, the default is to look for a tag named “latest” and use that. The tags generally specify versions (although this is not a requirement). [↩](https://training.play-with-docker.com/ops-s1-images/#fnref:1)
2. Type `vi index.js` then once the editor loads hit the `i` key. You can now type each of the commands as shown in the example. When you are finished hit the `<esc>` key then type `:wq` and that will save the file and take you back to the command prompt. You can type `ls` at the command prompt to ensure your *index.js* file is there or type `cat index.js` to make sure all the code is in the file. If you make a mistake in the editor and you have a hard time navigating the editor it might be easier to start fresh: simply type `<esc>` and then `:wq` if you are in the editor and then when you are back to the command line type `rm index.js` to delete the file and then start again. [↩](https://training.play-with-docker.com/ops-s1-images/#fnref:2)
3. Type `vi Dockerfile` then once the editor loads hit the `i` key. Type in each line of the Dockerfile code as shown in the example - capitalization is important! - then hit the `<esc>` key followed by `:wq`. To verify your Dockerfile exists and is correct type `cat Dockerfile`. If you make a mistake in the editor and you have a hard time navigating the editor it might be easier to start fresh: simply type `<esc>` and then `:wq` if you are in the editor and then when you are back to the command line type `rm Dockerfile` and then start again.

# Tasks

## Task 1. Testing the Docker installation
Start a container that runs the official hello-world image and check that everything functions correctly.

## Task 2. Running containers

1. Pull the busybox image from the official Docker registry to the local cache
1. Run a busybox container that executes the uptime command
1. Run an interactive busybox container; once you enter it, run the command wget google.com, then exit
1. Delete all containers and images created at the previous points

## Task 3. Building an image

1. go to the flask_app folder extracted from the file downloaded on D2L
1. modify the images array in the app.py file to make the application display other images of your choice
1. modify the templates/index.html file to display a different title related to your chosen images (i.e., change line 22)
1. build a Docker image entitled myflaskimage based on the provided Dockerfile
1. run a container based on your image on port 8888
1. check http://localhost:8888 to see if your container was started successfully

## Task 4

Now we make some changes to `index.js` and create a new file called `index2.js` with the code snippet below.

```javascript
var os = require("os");
var hostname = os.hostname();
var fs = require('fs');
var path = '/tmp/hostname'

if (fs.existsSync(path)) {
    // file exists
    console.log("hello from " + fs.readFileSync(path, 'utf8'));
} else {
    try {
        fs.writeFileSync(path, hostname);
        // file written successfully
        console.log("hello from " + hostname);
    } catch (err) {
        console.error(err);
    }
}
```
Here is a docker compose file example, which CANNOT be directly used. It serves as an exmaple to write your own docker compose file.
```yaml
version: "3"

services:
    api:
        build: . # builds the image from a Dockerfile
        image: register-image-name:version # uses an image from a registry
        environment:
            ENVIRONMENT_VARIABLE: value
        ports:
            - "5000:80"
        networks:
            - lab3-network

    postgres:
        image: postgres:12
        volumes:
            - lab3-volume:/var/lib/postgresql/data
            - ./scripts/init-db.sql:/docker-entrypoint-initdb.d/init-db.sql
        networks:
            - lab4-network

volumes:
    lab3-volume:

networks:
    lab3-network:
```

1. Write a YAML file `docker-compose.yaml` to boot up two services, `hello1` and `hello2` using the version `v0.2`. 
1. Use `docker compose build` to build the `hello` container with the source code file in the current directory.
1. Start and stop the docker compose configuration for multiple times using `docker compose up` and `docker compose down`. Check if the hashes of the services are the same in each run. Explain why.
1. Now define one volume for each of the two services. Then mount each volume to the path `/tmp` in the container. Repeat Step 2 and check if the hashes of the service are the same in each run. Explain why.


# Get your work checked by a TA

For each step in the task, the TA will check the completed files and/or outputs.